# Итоговое задание по Проекту 4. Компьютер говорит «Нет»
# ЮНИТ 5. ОСНОВНЫЕ АЛГОРИТМЫ МАШИННОГО ОБУЧЕНИЯ. ЧАСТЬ 1

# 1. Импорт библиотек и подготовка необходимых функций

In [ ]:

from pandas import Series
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import ttest_ind

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve

from sklearn import metrics

import datetime as DT


from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_curve 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


In [ ]:
# функция для анализа категориальных и номинативных переменных
def get_boxplot(column):
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=column, y='default', 
                data=data.loc[data.loc[:, column].isin(data.loc[:, column].value_counts().index[:])],
               ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [ ]:
RANDOM_SEED = 42
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# 2. Импорт данных

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
df_train = pd.read_csv(DATA_DIR+'/train.csv')
df_test = pd.read_csv(DATA_DIR+'test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
print('Размерность тренировочного датасета: ', df_train.shape)
display(df_train.head())
print('Размерность тестового датасета: ', df_test.shape)
display(df_test.head())

In [ ]:
data.info()

# Описание содержимого

1. client_id - идентификатор клиента

2. education - уровень образования

3. sex - пол заемщика

4. age - возраст заемщика

5. car - флаг наличия автомобиля

6. car_type - флаг автомобиля иномарки

7. decline_app_cnt - количество отказанных прошлых заявок

8. good_work - флаг наличия “хорошей” работы

9. bki_request_cnt - количество запросов в БКИ

10. home_address - категоризатор домашнего адреса

11. work_address - категоризатор рабочего адреса

12. income - доход заемщика

13. foreign_passport - наличие загранпаспорта

14. sna - связь заемщика с клиентами банка

15. first_time - давность наличия информации о заемщике

16. score_bki - скоринговый балл по данным из БКИ

17. region_rating - рейтинг региона

18. app_date - дата подачи заявки

19. default - флаг дефолта по кредиту

# 3. Предварительный анализ данных

In [ ]:
print(f'Всего в датасете {data.isnull().any().sum()} столбцов с пропущенными значениями.')
# посчитаем сколько колонок с пропущенными значениями

In [ ]:
data.isnull().sum()

В тренировочном датасете 73799 строк, в тестовом - 36349. В общей сложности представлены данные о 110 148 клиентах. Всего 19 предикторов, из них 1 - временной ряд, 6 бинарных, 7 категориальных и 5 числовых. Всего пропусков 478 (или 0,43 % от общего количества строк), все пропуски в переменной education.

Первое, что сделал - это удалил все колонки с нечисловыми значениями и попробовал запустить модель (model = LogisticRegression()). Score - 0.68488. Будем считать это отправной точкой.

In [ ]:
data.default.value_counts(normalize=True)

Но в целом, учитывая несбалансированность выборки можно попробовать всегда говорить, что клиент вернет деньги и мы будем правы в 90 процентах случаев.

In [ ]:
# Действие относится к самой простой модели, для которой данные вообще не обрабатывались! 
# data.drop(['app_date', 'education', 'sex','car','score_bki', 'car_type', 'good_work', 'foreign_passport'], axis='columns', inplace=True)

In [ ]:
# внесем данные из резюме в списки 
# временной ряд (1)
time_cols = ['app_date']
# бинарные переменные (default не включаем в список) (5+1 = 6)
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
# категориальные переменные (sample не включаем в список, так как мы сами его добавили) (7)
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']
# числовые переменные, client_id исключили из списка (5)
num_cols = ['age','decline_app_cnt','score_bki','bki_request_cnt','income']

# Обработка признаков

# 1. education

In [ ]:
data['education'].value_counts(dropna = False)

478 отсутвтвующих значения. Откуда они взялись - неизвестно, но полагаю, что люди без какого-либо вообще образования вряд-ли обращались за кредитом, поэтому попоробуем заполнить пропуски. Но, чтобы не заполнять их самым простым способом (самое распространенное значение) попробуем выявить некие взаимосвязи.

In [ ]:
# Визуализируем данные в столбце "education"
data['education'].value_counts(dropna = False).plot.bar()

Наибольшее количество клиентов имеет школьное образование. 

Визуализация взаимосвязи образования и значения нашей целевой переменной.

In [ ]:
data['education'][data['default'] == 1].hist(bins=50)

Ничего удивительного не обнаружилось, ввиду того, что наиболее распространенным значением является SCH, соответственно наиболее частые дефолты по кредитам встречаются именно с этой категорией клиентов.

Взаимосвязь образования и пола клиента.

In [ ]:
data['education'][data['sex'] == 'M'].hist(bins=50)

In [ ]:
data['education'][data['sex'] == 'F'].hist(bins=50)

Здесь можно увидеть, что среди женщин больше клиентов со школьным образованием.  

Зависимость образования и дохода.

In [ ]:
data['education'][data['income'] < data['income'].mean()].hist(bins=50)

Наибольшее количество клиентов с доходом меньше среднего имеют школьное образование.


Попробуем заполнить пропущенные значения с учетом полученных наблюдений.


In [ ]:
# заполняем пропуски для клиентов женского пола с доходм меньше среднего значением SCH.
data['education'] = np.where((data['sex'] == 'F') & (data['income'] < data['income'].mean()) & (data['education'].isna()), 'SCH', data['education'])

In [ ]:
# Проверим
data['education'].value_counts(dropna = False)

У нас осталось 281 пропущенное значение. Заполним их следующим по популярности значением.

In [ ]:
data['education'] = data['education'].fillna('GRD')

In [ ]:
# Проверим
data['education'].value_counts(dropna = False)

Пропущенных значений больше нет. 

#  2. region_rating

In [ ]:
data['region_rating'].value_counts().plot.bar()

Не понятно, что скрывается за этими числами, но предположу, что чем меньше число тем выше рейтинг региона проживания клиента.

In [ ]:
# создадим еще один столбец region_rating_freq - частота вхождения того или иного рейтинга в нашем дата-сете.
freq = data["region_rating"].value_counts(normalize = True)
data["region_rating_freq"] = data["region_rating"].map(freq)
data[["region_rating", "region_rating_freq"]].head(6)

# 3. app_date

In [ ]:
# Преобразуем формат признака
data.app_date = pd.to_datetime(data.app_date, format='%d%b%Y')

In [ ]:
# вытащим из даты день недели
a = []
for i in data.app_date:
    a.append(i.weekday())

In [ ]:
# создадим новый столбец
data['app_date_day'] = a
data['app_date_day']

In [ ]:
# замена значений в столбце app_date на числа.
data.app_date = data.app_date.astype(int)

In [ ]:
# проверим
data.app_date.value_counts()

In [ ]:
data.info()

# 3. sex 

Общеизвестный факт - женщины более аккуратны при пользовании кредитными продуктами. Попробуем это использовать.

In [ ]:
data['sex'][data['default'] == 1].hist(bins=50)

Gолучилось наоборот... потому, что количество клиентов женского пола болше чем мужского.

In [ ]:
data['sex'].value_counts()

In [ ]:
# оздадим новые признаки - предположим, что клиент женского пола, с доходом ниже среднего, с оценкой БКИ ниже среднего будет дефолтным.
data['female_default'] = 0
data['female_default'] = np.where((data['sex'] == 'F') & (data['income'] < data['income'].mean()) & (data['score_bki'] < data['score_bki'].mean()), '1', data['female_default'] )
data['female_default'] = data['female_default'].astype(int)

In [ ]:
# проверка
data.female_default.value_counts()

In [ ]:
data.info()

In [ ]:
# для клиентов мужского пола возьмем другие взаимосвязи - возраст меньше среднего, без хорошей работы.
data['male_default'] = 0
data['male_default'] = np.where((data['sex'] == 'M') & (data['age'] < data['age'].mean()) & (data['good_work'] == 0), '1', data['male_default'] )
data['male_default'] = data['male_default'].astype(int)

# 4. income

In [ ]:
data.income.value_counts()

In [ ]:
print(data.income.min())
print(data.income.max())

In [ ]:
# определение границ выбросов
median = data.income.median()
IQR = data.income.quantile(0.75) - data.income.quantile(0.25)
perc25 = data.income.quantile(0.25)
perc75 = data.income.quantile(0.75)
print('25-й перцентиль: {},'.format(perc25), '75-й перцентиль: {},'.format(perc75), "IQR: {}, ".format(IQR),"Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))

Логика подсказывает, что не стоит заменять данные с учетом предполагаемых выбросов. Столбец доходы может иметь совершенно разные значения.

In [ ]:
# Хотя я попробовал и так и так)
# data['income'] = np.where(data['income']<-22000.0,data['income'].median(), data['income'])

Создадим еще признаков из различных взаимосвязей

In [ ]:
# клиент с хорошей работой, с хорошей машиной предположительно платежеспособен
data['whithout_default'] = 0
data['whithout_default'] = np.where((data['good_work'] == 1) & (data['car_type'] =='Y') , '1', data['whithout_default'] )
data['whithout_default'] = data['whithout_default'].astype(int)

In [ ]:
# клиент с заграничным паспортом, с хорошей машиной также будет платежеспособен
data['whithout_default2'] = 0
data['whithout_default2'] = np.where((data['foreign_passport'] == 'Y') & (data['car_type'] =='Y') , '1', data['whithout_default2'] )
data['whithout_default2'] = data['whithout_default2'].astype(int)

In [ ]:
# клиент с отсутвием запросов в БКИ и отсутствием отказов, да и еще с хорошей работой платежеспособен
data['whithout_default3'] = 0
data['whithout_default3'] = np.where((data['decline_app_cnt'] == 0) & (data['bki_request_cnt'] == 0)& (data['income'] > data['income'].mean()) , '1', data['whithout_default3'] )
data['whithout_default3'] = data['whithout_default3'].astype(int)

# Подготовка данных для обучения.

In [ ]:
# Из курса известно что необходимо прологорифмировать значения отдельных признаков:

data['age'] = np.log(data['age'] + 1)
data['decline_app_cnt'] = np.log(data['decline_app_cnt'] + 1)
data['bki_request_cnt'] = np.log(data['bki_request_cnt'] + 1)
data['income'] = np.log(data['income'] + 1)

Для бинарных переменных используем LabelEncoder. # bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
data['sex'] = pd.Series(label_encoder.fit_transform(data['sex']))
print(dict(enumerate(label_encoder.classes_)))

In [ ]:
data['car'] = pd.Series(label_encoder.fit_transform(data['car']))
print(dict(enumerate(label_encoder.classes_)))

In [ ]:
data['car_type'] = pd.Series(label_encoder.fit_transform(data['car_type']))
print(dict(enumerate(label_encoder.classes_)))

In [ ]:
data['foreign_passport'] = pd.Series(label_encoder.fit_transform(data['foreign_passport']))
print(dict(enumerate(label_encoder.classes_)))

Значений типа object в нашем датасете больше нет (кроме app_date).

Проверим на сколько изменится score.

score 0.72755

In [ ]:
# реализуем метод OneHotLabels через get_dummies для категориальных признаков
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time', 'app_date_day']

data=pd.get_dummies(data, prefix=cat_cols, columns=cat_cols)

In [ ]:
data.info()

In [ ]:
# для того чтобы выбрать значимые признаки воспользуемся следуещей функцией:
def get_stat_dif(column):
    cols = data.loc[:, column].value_counts().index[:]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(data.loc[data.loc[:, column] == comb[0], 'default'], data.loc[data.loc[:, column] == comb[1], 'default']).pvalue <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break

In [ ]:
list(data.columns.values)

In [ ]:
cat_cols

In [ ]:
bin_cols

In [ ]:
for cols in ['education_ACD',
 'education_GRD',
 'education_PGR',
 'education_SCH',
 'education_UGR','region_rating_20',
 'region_rating_30',
 'region_rating_40',
 'region_rating_50',
 'region_rating_60',
 'region_rating_70',
 'region_rating_80','home_address_1',
 'home_address_2',
 'home_address_3',
 'work_address_1',
 'work_address_2',
 'work_address_3', 'sna_1',
 'sna_2',
 'sna_3',
 'sna_4',
 'first_time_1',
 'first_time_2',
 'first_time_3',
 'first_time_4', 'female_default', 'whithout_default', 'whithout_default2', 'male_default', 'whithout_default3', 'app_date_day_0','app_date_day_1','app_date_day_2','app_date_day_3','app_date_day_4','app_date_day_5','app_date_day_6' ]:
    get_stat_dif(cols)
 

Удаление столбцов не влияющих на нашу целевую переменную в конце подготовки данных

In [ ]:
# Анализ уровня значимости категориальных и бинарных признаков
all_cat_and_bin_cols = ['education_ACD',
 'education_GRD',
 'education_PGR',
 'education_SCH',
 'education_UGR','region_rating_20',
 'region_rating_30',
 'region_rating_40',
 'region_rating_50',
 'region_rating_60',
 'region_rating_70',
 'region_rating_80','home_address_1',
 'home_address_2',
 'home_address_3',
 'work_address_1',
 'work_address_2',
 'work_address_3', 'sna_1',
 'sna_2',
 'sna_3',
 'sna_4',
 'first_time_1',
 'first_time_2',
 'first_time_3',
 'first_time_4', 'sex', 'car', 'car_type', 'good_work', 'foreign_passport', 'female_default','male_default', 'whithout_default', 'whithout_default2', 'whithout_default3', 'app_date_day_0','app_date_day_1','app_date_day_2','app_date_day_3','app_date_day_4','app_date_day_5','app_date_day_6']
temp_df = data[data['sample']==1]
imp_cat = pd.Series(mutual_info_classif(temp_df[all_cat_and_bin_cols], temp_df['default'], discrete_features =True), index = all_cat_and_bin_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh', title = 'Значимость бин. и категор. переменных по Mutual information test')

Наиболее значимым признаком среди категориальных и бинарных является sna - связь заемщика с клиентами банка.

In [ ]:
# Анализ уровня значимости для числовых признаков
imp_num = Series(f_classif(data[num_cols], data['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
list(data.columns.values)

In [ ]:
data.drop(['app_date_day_0','app_date_day_1','app_date_day_2','app_date_day_3','app_date_day_5','app_date_day_6','education_UGR', 'first_time_3', 'car','sna_3','sna_2', 'education_SCH','education_GRD', 'region_rating_60','region_rating_50', 'home_address_1', 'work_address_3','work_address_1', 'work_address_2'], axis='columns', inplace=True)

In [ ]:
num_cols = ['age','decline_app_cnt','score_bki','bki_request_cnt','income', 'region_rating_freq']


In [ ]:
#Стандартизация числовых переменных
data[num_cols] = StandardScaler().fit_transform(data[num_cols].values)

In [ ]:
plt.rcParams['figure.figsize'] = (30,20)
sns.heatmap(data.corr().round(2), annot = True) 

Учитывая что наша выборка является несбалансированной представляется целесообразным уменьшить количество объектов превалирующих классов (undersampling)

In [ ]:
sns.countplot(x='default', data=data)

In [ ]:
# Найдем количество клиентов, которые являются превалирующим классом
frauds = len(data[data['default'] == 1])

In [ ]:
# возьмем индексы клиентов - не превалирующего класса
non_fraud_indices = data[data.default == 0].index


In [ ]:
# Индексы случайной выборки не связанные с превалирующим классом
random_indices = np.random.choice(non_fraud_indices,frauds, replace=False)

In [ ]:
# найдем индексы превалирующего класса
fraud_indices = data[data.default == 1].index

In [ ]:
# Объединим
under_sample_indices = np.concatenate([fraud_indices,random_indices])

In [ ]:
under_sample = data.loc[under_sample_indices]

In [ ]:
sns.countplot(x='default', data=under_sample)

После применения undersampling score  составил 0.73084

In [ ]:
# Разделение данных на train an test. 
# разделение и дальнейшее обучение на сбалансированной выборке показало не самый лучший результат с учетом 
# выбранной метрики. Так самое лучшее, что удалось добиться это 49 место на Kaggle.

#train_data = under_sample.query('sample == 1').drop(['sample', 'client_id'], axis=1)

train_data = data.query('sample == 1').drop(['sample', 'client_id'], axis=1)

test_data = data.query('sample == 0').drop(['sample', 'client_id'], axis=1)

In [ ]:
# Действие относится к первым 3 попыткам.
#train_data = data.query('sample == 1').drop(['sample', 'client_id'], axis=1)
#test_data = data.query('sample == 0').drop(['sample', 'client_id'], axis=1)

In [ ]:
X_train=train_data.drop(['default'], axis=1)
y_train = train_data.default.values
X_test = test_data.drop(['default'], axis=1)
y_test = test_data.default.values

In [ ]:
test_data.shape, train_data.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# Здесь были опробованы различные модели RandomForestClassifier, GradientBoostingClassifier, KNeighborsClassifier,
# GaussianNB, LogisticRegression. Но наилучший результат Score = 0.73621 получился с GradientBoostingClassifier.
model = GradientBoostingClassifier(learning_rate=0.2, min_samples_leaf = 4, max_features=0.4)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_train)

In [ ]:
# confusion_matrix
conf_mat = metrics.confusion_matrix(y_train, preds)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

Судя по данным confusion_matrix при применении несбалансированной выборки у нас 9 тысяч ложно отрицательных значения. (дефолт классифицирован как не-дефолт). Что критично для бизнеса, ввиду значительных возможных финансовых потерь.



После обучения модели на сбалансированной выборке количество ложно отрицательных значений снизилось до 1тыс., зато увеличилось количество лолжно положительных значений
(не-дефолт классифицирован как дефолт). Данная ситуация является более предпочтительной ввиду того, что в ситуации когда кредит не выдан клиенту который бы вернул деньги банк теряет прибыл в видете процента по кредиту. В ситуации же когда кредит выдается дефолтному клиенту банк теряет 100% выданного кредита.

In [ ]:
pred_prob = model.predict_proba(X_train)

In [ ]:
preds = pred_prob[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_train, preds)
roc_auc = metrics.auc(fpr, tpr)


In [ ]:
# method I: plt

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()



In [ ]:
y_pred_prob = model.predict_proba(X_test)[:,1]


submit = pd.DataFrame(df_test.client_id)
submit['default']=y_pred_prob
submit.to_csv('submission.csv', index=False)